In [22]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier, Pool
#from sklearn.metrics import f1_score, make_scorer
#from optuna import Trial
#from optuna.samplers import TPESampler
#from sklearn.model_selection import StratifiedKFold,train_test_split

In [23]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [24]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [25]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [26]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [27]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [32]:
def model_fit_predict(dataset, targetdata, testset):
    model1 = RandomForestClassifier()
    model2 = XGBClassifier(n_estimators=2000, learning_rate=0.03, max_depth=20)
    model3 = LGBMClassifier(n_estimators=2000, max_depth=20)
    model4 = CatBoostClassifier(iterations=2, max_depth=7, learning_rate=0.03, 
                              loss_function='Logloss', verbose=True)

    # fit
    model1.fit(train_x,train_y)
    model2.fit(train_x,train_y)
    model3.fit(train_x,train_y)
    model4.fit(train_x,train_y)
    
    # predict
    pred1 = model1.predict(test_x)
    pred2 = model2.predict(test_x)
    pred3 = model3.predict(test_x)
    pred4 = model4.predict(test_x)

    pred_sum = pred1 + pred2 + pred3 + pred4
    
      # voting ensemble
    predict_final = []
    for pred in pred_sum:
        if pred > 2:
            predict_final.append(1)
        else:
            predict_final.append(0)

    return np.array(predict_final)


# catboost = CatBoostClassifier(max_depth =7,random_state=37).fit(train_x, train_y)
print('Done.')

Done.


In [34]:
model1_pred = model_fit_predict(train_x, train_y, test_x)

NameError: name 'XGBClassifier' is not defined

In [10]:
# 함수 생성 
def objective(trial):
    X = train_x
    y = train_df['Y_Class']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36)

    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = CatBoostClassifier(**param)

    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

In [11]:
import optuna

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-03-20 21:33:20,429] A new study created in memory with name: no-name-600652f5-6e04-477a-8e92-f0bc0043ebed
[W 2023-03-20 21:33:21,050] Trial 0 failed with parameters: {'learning_rate': 0.33723163454692767, 'objective': 'Logloss', 'colsample_bylevel': 0.06650259306251527, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.28315265716539845} because of the following error: CatBoostError('C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/target_converter.cpp:379: Target with classes must contain only 2 unique values for binary classification').
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_10476\95304098.py", line 27, in objective
    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/target_converter.cpp:379: Target with classes must contain only 2 unique values for binary classification

In [ ]:
submit = pd.read_csv('./submit.csv')

In [ ]:
submit['Y_Class'] = preds_catboost
plt.hist(preds_catboost)

In [ ]:
submit.to_csv('./submission.csv', index=False)